In [3]:
!pip install texthero

You should consider upgrading via the 'c:\users\admin\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [3]:
import os
import pandas as pd

In [4]:
input_file = '.\labeled_data_corpus.csv'
if not os.path.exists(input_file):
    raise RuntimeError(f'No input file: {input_file}')

df = pd.read_csv(input_file)
train_df = df[df['subset'] == 'train']
test_df = df[df['subset'] == 'test']
print('num rows for train: %d', train_df.shape[0])

num rows for train: %d 5233


In [5]:
X_train = train_df['msg']
y_train = train_df['label']

X_test = test_df['msg']
y_test = test_df['label']

In [7]:
X_train[:10]

0    здравствуйте. ишу 2х спальную квартиру в лимас...
1    #сниму  комнату в лимассоле или недалеко от не...
2                        мошенник риэлторским услугам.
3    **sales**    reg.1053 lic.489/e **stylish apar...
4    важно: [valerii korol](tg://user?id=193474890)...
5    аренда  no: 367/e ️ларнака️между пила и декели...
6    привет  ищу виллу посуточно с бюджетом 2000€ в...
7    важно: [liss](tg://user?id=202814885), если ты...
8                               total messages: 126772
9    аренда  ️ларнака ️в центре города️ saint lazar...
Name: msg, dtype: object

In [6]:
from numpy import asarray
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import numpy as np
from sklearn.model_selection import ParameterGrid

# определяем датасет
X_train = train_df['msg'].values
y_train = train_df['label']

X_test = test_df['msg'].values
y_true = test_df['label']
# и нашу модель вместе с кросс-валидацией
model = XGBClassifier()
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X_train, y_train, scoring='accuracy', cv=cv, n_jobs=-1, error_score='raise')
print('Точность: %.3f (%.3f)' % (mean(n_scores), std(n_scores)))
# тренируем модель на всём наборе данных



NameError: name 'n' is not defined

In [ ]:
model = XGBClassifier()
model.fit(X_test, y_test)
# предсказываем
row = [2.56999479, -0.13019997, 3.16075093, -4.35936352, -1.61271951, -1.39352057, -2.48924933, -1.93094078, 3.26130366, 2.05692145]
row = asarray(row).reshape((1, len(row)))
yhat = model.predict(row)
print('Предсказание (Предикт): %d' % yhat[0])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score

import numpy as np
from sklearn.model_selection import ParameterGrid

X_train = train_df['msg'].values
y_train = train_df['label']

X_test = test_df['msg'].values
y_true = test_df['label']

grid = {
    'max_df': np.arange(0.01, 1.0, 0.01),
    'min_df': np.arange(1, 20, 1),
}

best_params = {'max_df': None, 'min_df': None}
best_score = 0.0

num_iters = len(ParameterGrid(grid))

cnt = 0
for param_values in ParameterGrid(grid):
    # fit 
    vectorizer = TfidfVectorizer(**param_values).fit(X_train)
    X_train_csr = vectorizer.transform(X_train)
    lr = LogisticRegression().fit(X_train_csr, y_train)
    # predict
    X_test_csr = vectorizer.transform(X_test)
    y_pred = lr.predict(X_test_csr)
    cur_score = f1_score(y_true, y_pred)
    if cur_score > best_score:
        best_score = cur_score
        best_params.update(param_values)
    if cnt % 250 == 0:
        logging.info(
            'iteration: %d of %d; %s; best_score= %.4f',
            cnt, num_iters, param_values, best_score
        )
    cnt = cnt + 1
print('best params: %s, best_score %.4f', best_params, best_score)